In [34]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Define layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)

        # Use the calculated size for the fully connected layer
        self.fc = nn.Linear(48640, 2)

    def forward(self, x):
        # Forward pass through the conv and pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Flatten the tensor for the fully connected layer
        x = x.view(x.size(0), -1)
        # Forward pass through the fully connected layer
        x = self.fc(x)
        return x


In [39]:
# dimensions of images 160 × 155 pixels
print("Script started, loading data...")

transform = transforms.Compose([
    transforms.Resize((155, 160)),  # Resize all images to 155x160
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# train_dataset = datasets.ImageFolder(root='Data/train', transform=transform)
# val_dataset = datasets.ImageFolder(root='Data/validate', transform=transform)
test_dataset = datasets.ImageFolder(root='./Data/train', transform=transform)

# train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)


model = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
print("Starting training...")
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(test_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(test_loader)}], Loss: {loss.item():.4f}')

Script started, loading data...
Starting training...
Epoch [1/100], Step [10/52], Loss: 0.4456
Epoch [1/100], Step [20/52], Loss: 0.6300
Epoch [1/100], Step [30/52], Loss: 0.4236
Epoch [1/100], Step [40/52], Loss: 0.2345
Epoch [1/100], Step [50/52], Loss: 0.3103
Epoch [2/100], Step [10/52], Loss: 0.0508
Epoch [2/100], Step [20/52], Loss: 0.2924
Epoch [2/100], Step [30/52], Loss: 0.2153
Epoch [2/100], Step [40/52], Loss: 0.3066
Epoch [2/100], Step [50/52], Loss: 0.2271
Epoch [3/100], Step [10/52], Loss: 0.2272
Epoch [3/100], Step [20/52], Loss: 0.2082
Epoch [3/100], Step [30/52], Loss: 0.1025
Epoch [3/100], Step [40/52], Loss: 0.1174
Epoch [3/100], Step [50/52], Loss: 0.2128
Epoch [4/100], Step [10/52], Loss: 0.1240
Epoch [4/100], Step [20/52], Loss: 0.2262
Epoch [4/100], Step [30/52], Loss: 0.1897
Epoch [4/100], Step [40/52], Loss: 0.0485
Epoch [4/100], Step [50/52], Loss: 0.0301
Epoch [5/100], Step [10/52], Loss: 0.0712
Epoch [5/100], Step [20/52], Loss: 0.2558
Epoch [5/100], Step [30

In [40]:
model.eval()
class_labels = {0: 'No Glasses', 1: 'Glasses'}

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Returns the max value of each row of the input tensor in the given dimension dim
        for i in range(images.size(0)):  # Iterate over each image in the batch
            print(f'Image {i+1}: {class_labels[predicted[i].item()]}')
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

Image 1: Glasses
Image 2: Glasses
Image 3: No Glasses
Image 4: No Glasses
Image 5: Glasses
Image 6: Glasses
Image 7: Glasses
Image 8: Glasses
Image 9: No Glasses
Image 10: Glasses
Image 1: No Glasses
Image 2: Glasses
Image 3: Glasses
Image 4: No Glasses
Image 5: Glasses
Image 6: Glasses
Image 7: Glasses
Image 8: Glasses
Image 9: Glasses
Image 10: No Glasses
Image 1: Glasses
Image 2: Glasses
Image 3: No Glasses
Image 4: No Glasses
Image 5: Glasses
Image 6: No Glasses
Image 7: No Glasses
Image 8: No Glasses
Image 9: Glasses
Image 10: Glasses
Image 1: Glasses
Image 2: Glasses
Image 3: Glasses
Image 4: Glasses
Image 5: No Glasses
Image 6: Glasses
Image 7: No Glasses
Image 8: No Glasses
Image 9: Glasses
Image 10: Glasses
Image 1: Glasses
Image 2: Glasses
Image 3: Glasses
Image 4: Glasses
Image 5: Glasses
Image 6: Glasses
Image 7: Glasses
Image 8: Glasses
Image 9: Glasses
Image 10: Glasses
Image 1: Glasses
Image 2: Glasses
Image 3: No Glasses
Image 4: Glasses
Image 5: Glasses
Image 6: Glasse

In [41]:
# Load the image
image_path = 'Data/test/face-1.png'  # Replace with the path to your image
image = Image.open(image_path).convert('RGB')

# Apply the same transformations as used for your training data
transform = transforms.Compose([
    transforms.Resize((155, 160)),  # Resize the image
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image = transform(image)

# Add an extra dimension to the image tensor (to represent batch size of 1)
image = image.unsqueeze(0)  # Converts from (C, H, W) to (1, C, H, W)

# Load your trained model (ensure it is in evaluation mode)
model = CNN()  # Assuming your model is named CNN
model.eval()

# Predict
with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs.data, 1)
    class_labels = {0: 'No Glasses', 1: 'Glasses'}
    prediction = class_labels[predicted.item()]

print(f'The person in the image is predicted to have: {prediction}')

The person in the image is predicted to have: Glasses
